In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

Read the data and split them to in time and out of time

In [2]:
inTime = pd.read_csv('finalVariable_inTimeData3.csv')
outTime = pd.read_csv('finalVariable_outTimeData3.csv')

outTime = outTime.iloc[:,1:]
inTime = inTime.iloc[:,1:]

inTimeX = inTime.drop('FraudLabel', axis=1)
inTimeY = inTime.loc[:,'FraudLabel']
inTimeX.head(5)
#inTimeY.head(5)

,address-zip5-homephone_count30_date,address-zip5_0_count_address-zip5_30_count_Ave,address-zip5_0_count_address-zip5_7_count_Ave,address-zip5_count0_date,address-zip5_count180_date,address-zip5_count1_date,address-zip5_count30_date,address-zip5_count3_date,address-zip5_count7_date,address-zip5_count90_date,...,address_count1_date,address_count30_date,address_count3_date,address_count7_date,address_count90_date,name-dob_count180_date,name-dob_count30_date,name-dob_count90_date,ssn-dob_count180_date,ssn-dob_count30_date
0,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,30.0,7.0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


split in time into train and test

In [3]:
from sklearn.model_selection import train_test_split
Y = inTimeY
X = inTimeX
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, )


x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

fit the adaboosting model

In [16]:
## train table
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4, min_samples_leaf = 8),
                                        n_estimators=100,loss='linear')

regr.fit(x_train, y_train)
PredictTrainY = regr.predict(x_train)
PredictTrainY = pd.Series(PredictTrainY)
temp = pd.DataFrame(columns = ["PredictTrainY","RealY"])
temp["PredictTrainY"] = PredictTrainY
temp["RealY"] = y_train
temp.sort_values(by = "PredictTrainY", ascending = False, inplace = True)
Table = pd.DataFrame(columns = ["FDR_Rate", "NumRecords", "NumBad","FDR"])
for FDR_Rate in range(1,21):
    FDR_Rate = FDR_Rate/100
    Percent = int(len(x_train) *FDR_Rate)
    FDR = temp["RealY"][:Percent].sum() / y_train.sum()   
    training_NumberRecords = Percent
    training_NumberBades = temp["RealY"][:Percent].sum()
    Table = Table.append({'FDR_Rate': FDR_Rate, 'NumRecords': training_NumberRecords,'NumBad': training_NumberBades, "FDR":FDR}, ignore_index=True)
Table
Table.to_excel('Train_Table.xlsx')



In [13]:
##test table
PredictTestY = regr.predict(x_test)
PredictTestY = pd.Series(PredictTestY)
temp = pd.DataFrame(columns = ["PredictTestY","RealY"])
temp["PredictTestY"] = PredictTestY
temp["RealY"] = y_test
temp.sort_values(by = "PredictTestY", ascending = False, inplace = True)
Table = pd.DataFrame(columns = ["FDR_Rate", "NumRecords", "NumBad"])
for FDR_Rate in range(1,21):
    FDR_Rate = FDR_Rate/100
    Percent = int(len(x_test) *FDR_Rate)
    FDR = temp["RealY"][:Percent].sum() / y_test.sum()   
    NumberRecords = Percent
    NumberBades = temp["RealY"][:Percent].sum()
    Table = Table.append({'FDR_Rate': FDR_Rate, 'NumRecords': NumberRecords,'NumBad': NumberBades}, ignore_index=True)
Table
Table.to_excel('Test_Table.xlsx')

In [14]:
## OOT table
outTimeX = outTime.drop('FraudLabel', axis=1)
outTimeY = outTime.loc[:,'FraudLabel']

PredictedY = regr.predict(outTimeX)
PredictedY = pd.Series(PredictedY)

temp = pd.DataFrame(columns = ["PredictedY","RealY"])
temp["PredictedY"] = PredictedY
temp["RealY"] = outTimeY
temp.sort_values(by = "PredictedY", ascending = False, inplace = True)

Table = pd.DataFrame(columns = ["FDR_Rate", "NumRecords", "NumBad"])
for FDR_Rate in range(1,21):
    FDR_Rate = FDR_Rate/100
    Percent = int(len(outTimeX) *FDR_Rate)
    FDR = temp["RealY"][:Percent].sum() / y_test.sum()   
    NumberRecords = Percent
    NumberBades = temp["RealY"][:Percent].sum()
    Table = Table.append({'FDR_Rate': FDR_Rate, 'NumRecords': NumberRecords,'NumBad': NumberBades}, ignore_index=True)
Table
Table.to_excel('OOT_Table.xlsx')